<a href="https://colab.research.google.com/github/swapnilmn/Assignment_03/blob/main/Question_5_Ass_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1 (15 Marks)
Build a RNN based seq2seq model which contains the following layers: (i) input layer for character embeddings (ii) one encoder RNN which sequentially encodes the input character sequence (Latin) (iii) one decoder RNN which takes the last state of the encoder as input and produces one output character at a time (Devanagari). 


#Librares

In [ ]:
import csv
import numpy as np
import random
import torch
import torch.nn as nn 
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!pip install wandb -qU
import wandb
wandb.login()
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 783.0 kB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Mounted at /content/drive


# Load of data

In [ ]:
import math  # Import the math module for mathematical operations
import time  # Import the time module for time-related functions
import torch  # Import the torch module for working with tensors
import numpy as np  # Import the numpy module for numerical operations

def asMinutes(s):
    # Convert seconds to minutes and seconds
    m = math.floor(s / 60)  # Calculate the number of minutes
    s -= m * 60  # Subtract the minutes converted to seconds from the total seconds
    return '%dm %ds' % (m, s)  # Return the formatted string for minutes and seconds

def span(since, percent):
    # Calculate the elapsed time and estimated remaining time
    now = time.time()  # Get the current time
    s = now - since  # Calculate the elapsed time in seconds
    es = s / (percent)  # Calculate the estimated total time in seconds
    rs = es - s  # Calculate the remaining time in seconds
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))  # Return the formatted string for elapsed time and remaining time

def key_presenting(dictionary, values):
    # Find keys in the dictionary that match the given values
    found_keys = []  # Initialize an empty list to store the found keys
    for value in values:  # Iterate over the given values
        for key, val in dictionary.items():  # Iterate over the dictionary items
            if value == val:  # If the value matches the dictionary value
                found_keys.append(key)  # Add the key to the list of found keys
    
    if found_keys:  # If any keys are found
        str = ''.join(found_keys)  # Concatenate the found keys into a single string
        if str[-1] == '\n':  # If the string ends with a newline character
            str = str[:-1]  # Remove the newline character from the end of the string
        elif str[0] == '\t':  # If the string starts with a tab character
            str = str[1:]  # Remove the tab character from the start of the string
    else:  # If no keys are found
        print("No keys found for the given values.")  # Print a message indicating no keys were found
    
    return str  # Return the string containing the found keys

def char_acc(targets, outputs):
    # Calculate character-level accuracy
    with torch.no_grad():  # Disable gradient calculation for efficiency
        count = 0  # Initialize a counter for correct predictions
        total_count = 0  # Initialize a counter for total predictions
        for i in range(targets.shape[0]):  # Iterate over the rows of the targets tensor
            same_elements = []  # Initialize an empty list to store the comparison results
            for j in range(targets.shape[1]):  # Iterate over the columns of the targets tensor
                if targets[i][j] != 67 or targets[i][j] != 66:  # Check if the target value is not equal to 67 or 66
                    same_elements.append(outputs[i][j].item() == targets[i][j].item())  # Append the comparison result to the list
            count += np.sum(same_elements)  # Add the number of True values in the list to the count
            total_count += len(same_elements)  # Add the total number of elements in the list to the total count
    return count / total_count  # Return the character-level accuracy as a ratio of correct predictions to total predictions

def word_acc(targets, outputs):
    # Calculate word-level accuracy
    outputs1 = torch.argmax(outputs, dim=1)  # Get the index of the maximum value along the second dimension of the outputs tensor
    with torch.no_grad():  # Disable gradient calculation for efficiency
        count = 0  # Initialize a counter for correct predictions
        for i in range(targets.shape[0]):  # Iterate over the rows of the targets tensor
            if (outputs1[i] == targets[i]).sum().item() == targets.shape[1]:  # Check if all elements in the row are equal
                count = count + 1  # Increment the count if all elements are equal
    return count / targets.shape[0]  # Return the word-level accuracy as a ratio of correct predictions to total predictions

def equi_points(data, epochs):
    # Sample equidistant points from data
    step = len(data) // epochs  # Calculate the step size to sample equidistant points
    indices = np.arange(0, len(data), step)  # Generate indices using numpy arange function
    equidistant_points = [data[i] for i in indices]  # Extract the equidistant points from the data list
    
    return equidistant_points  # Return the list of equidistant points


In [ ]:
import csv

def read_csv_files():
    # Define the file paths for the CSV files
    file_paths = {
        'file1': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_test.csv',  # Path to 'mar_test.csv'
        'file2': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_valid.csv',  # Path to 'mar_valid.csv'
        'file3': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_train.csv'  # Path to 'mar_train.csv'
    }

    data = {}  # Dictionary to store the CSV data

    for file_key, file_path in file_paths.items():
        with open(file_path) as file:
            csv_reader = csv.reader(file)
            header = next(csv_reader)  # Get the header row of the CSV
            rows = [row for row in csv_reader]  # Get all the data rows of the CSV
            data[file_key] = {
                'header': header,
                'rows': rows
            }

    return data

# Execute the function to read CSV files
csv_data = read_csv_files()

# Access the data
test = csv_data['file1']['rows']  # Contains rows from 'mar_test.csv'
val = csv_data['file2']['rows']  # Contains rows from 'mar_valid.csv'
train = csv_data['file3']['rows']  # Contains rows from 'mar_train.csv'


# Data Preprocessing

In [ ]:
# Define a function to read data from a list of pairs
def Data_Reading(list):
  i = []  # List to store the first element of each pair
  t = []  # List to store the second element of each pair
  for pair in list:
    i.append(pair[0])  # Append the first element to 'i'
    t.append(pair[1])  # Append the second element to 't'
  return i, t  # Return the two lists

# Read data from the 'train' list using the 'Data_Reading' function
train_inputs, train_targets = Data_Reading(train)

# Read data from the 'test' list using the 'Data_Reading' function
test_inputs, test_targets = Data_Reading(test)

# Read data from the 'val' list using the 'Data_Reading' function
val_inputs, val_targets = Data_Reading(val)

# Print the second element of the 'train_inputs' list
# print(train_inputs[1])

# Print the second element of the 'train_targets' list
# print(train_targets[1])

# Define a dictionary to store variable values
variable_dict = {
    'sc': '\t',    # 'sc' represents a tab character
    'eb': '\n',    # 'eb' represents a newline character
    'bc ': ' ',    # 'bc' represents a space character
    'unc': '\r'    # 'unc' represents a carriage return character
}

# Accessing the values from the dictionary
sc = variable_dict['sc']    # Assign the value of '\t' to 'sc'
eb = variable_dict['eb']    # Assign the value of '\n' to 'eb'
bc  = variable_dict['bc ']  # Assign the value of ' ' to 'bc'
unc = variable_dict['unc']  # Assign the value of '\r' to 'unc'


# Function to create dictionaries for language encoding

def dictlang(inputs, targets):
    dict = {}  # Dictionary for input language
    mil = 0  # Maximum input language length
    ichar = []  # List of characters in input language

    dict_t = {}  # Dictionary for target language
    mtl = 0  # Maximum target language length
    tchar = []  # List of characters in target language

    for s in inputs:
        mil = max(len(s), mil)
        for char in s:
            if char not in dict:
                dict[char] = len(ichar)
                ichar.append(char)

    # Add special characters if they are not present in the input language dictionary
    if bc not in dict:
        dict[bc] = len(ichar)
        ichar.append(bc)

    dict[unc] = len(ichar)
    ichar.append(unc)

    # Add special characters if they are not present in the target language dictionary
    if sc not in dict_t:
        dict_t[sc] = len(tchar)
        tchar.append(sc)

    for s in targets:
        mtl = max(len(s) + 2, mtl)
        for char in s:
            if char not in dict_t:
                dict_t[char] = len(tchar)
                tchar.append(char)

    # Add special characters if they are not present in the target language dictionary
    if eb not in dict_t:
        dict_t[eb] = len(tchar)
        tchar.append(eb)

    # Add special characters if they are not present in the target language dictionary
    if bc not in dict_t:
        dict_t[bc] = len(tchar)
        tchar.append(bc)

    return dict, mil, ichar, dict_t, mtl, tchar

# Create dictionaries for language encoding using training, validation, and test inputs and targets
dict, mil, ichar, dict_t, mtl, tchar = dictlang(train_inputs + val_inputs + test_inputs, train_targets + val_targets + test_targets)


def encoding_w(a, d, ml, l):
    encs = []  # List to store encoded words
    for word in a:
        enc = []  # List to store encoded characters of a word
        for char in word:
            if char in d:
                enc.append(d[char])  # Encode character if present in the dictionary
            else:
                enc.append(d[unc])  # Encode unknown character
        if (l == 0):
            while len(enc) < ml:
                enc.append(d[bc])  # Pad with blank character if required
        if (l == 1):
            enc.insert(0, d[sc])  # Insert start character at the beginning
            while len(enc) < ml - 1:
                enc.append(d[bc])  # Pad with blank character if required
            enc.append(d[eb])  # Append end character at the end
        encs.append(enc)  # Add encoded word to the list
    return encs

def tokenize(train,val,test,dict,dict_t,mil,mtl):
    # Tokenize the data for training, validation, and testing sets
    ti, tt = Data_Reading(train)  # Read training data
    tti, ttt = Data_Reading(test)  # Read testing data
    vi, vt = Data_Reading(val)  # Read validation data

    # Encode the training, validation, and testing data
    e_t_i = encoding_w(ti,dict,mil,0)  # Encode training input data
    e_t_t = encoding_w(tt,dict_t,mtl,1)  # Encode training target data
    e_v_i = encoding_w(vi,dict,mil,0)  # Encode validation input data
    e_v_t = encoding_w(vt,dict_t,mtl,1)  # Encode validation target data
    e_tt_i = encoding_w(tti,dict,mil,0)  # Encode testing input data
    e_tt_t = encoding_w(ttt,dict_t,mtl,1)  # Encode testing target data

    return e_t_i, e_t_t, e_v_i, e_v_t, e_tt_i, e_tt_t

# Tokenize the data using the given parameters
e_t_i, e_t_t, e_v_i, e_v_t, e_tt_i, e_tt_t = tokenize(train, val, test, dict, dict_t, mil, mtl)

# Generate a random number within the range [0, 100]
r = random.randint(0, 100)

# Print the keys corresponding to the values of the encoded data at index r
# print(key_presenting(dict, e_t_i[int(r)]))
# print(key_presenting(dict_t, e_t_t[int(r)]))


# Define a function to convert input and target data into tensor pairs
def tensor_conversion(ti, tg):
    pr = []
    for id, td in zip(ti, tg):
        # Convert each item in ti to a torch tensor
        it = torch.tensor(id)
        # Convert each item in tg to a torch tensor
        tt = torch.tensor(td)
        # Append the tensor pair to the result list
        pr.append((it, tt))
    return pr

# Convert e_t_i and e_t_t to tensor pairs
e_t_p = tensor_conversion(e_t_i, e_t_t)

# Convert e_v_i and e_v_t to tensor pairs
e_v_p = tensor_conversion(e_v_i, e_v_t)

# Re-assign e_t_p with new tensor pairs converted from e_tt_i and e_tt_t
e_t_p = tensor_conversion(e_tt_i, e_tt_t)

# Create a tuple containing e_t_p, e_v_p, and e_t_p
pairs = (e_t_p, e_v_p, e_t_p)

# Randomly choose a pair from e_t_p
pair = random.choice(e_t_p)

# Print the keys in dict corresponding to the values in pair[0]
# print(key_presenting(dict, pair[0]))

# Print the keys in dict_t corresponding to the values in pair[1]
# print(key_presenting(dict_t, pair[1]))


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(this, device, cell_type, vocab_size, embed_dim, hidden_size, num_layers=1, bidirectional=False, dropout_p=0):
        super(EncoderRNN, this).__init__()
        this.embedding = nn.Embedding(vocab_size, embed_dim)  # Embedding layer for converting input indices to dense vectors.
        this.hidden_size = hidden_size  # Dimensionality of the hidden state of the RNN.
        this.num_layers = num_layers  # Number of recurrent layers.
        this.bidirectional = bidirectional  # Flag indicating whether the RNN is bidirectional.
        this.cell_type = cell_type  # Type of the RNN cell: 'lstm', 'rnn', or 'gru'.
        this.dropout_p = dropout_p  # Dropout probability.
        this.dropout = nn.Dropout(this.dropout_p)  # Dropout layer for regularization.
        if cell_type == 'lstm':
            this.rnn = nn.LSTM(embed_dim, hidden_size, num_layers=this.num_layers, batch_first=True, dropout=dropout_p, bidirectional=this.bidirectional)  # LSTM RNN module.
        elif cell_type == 'rnn':
            this.rnn = nn.RNN(embed_dim, hidden_size, num_layers=this.num_layers, batch_first=True, dropout=dropout_p, bidirectional=this.bidirectional)  # Vanilla RNN module.
        elif cell_type == 'gru':
            this.rnn = nn.GRU(embed_dim, hidden_size, num_layers=this.num_layers, batch_first=True, dropout=dropout_p, bidirectional=this.bidirectional)  # GRU module.

    def forward(this, x, hidden, cell):
        out = this.embedding(x)  # Convert input indices to dense vectors.
        out = this.dropout(out)  # Apply dropout to the input.
        if this.cell_type == 'lstm':
            out, (hidden, cell) = this.rnn(out, (hidden, cell))  # Forward pass through the LSTM RNN.
            return out, hidden, cell  # Return the output, hidden state, and cell state.
        elif this.cell_type == 'rnn':
            out, hidden = this.rnn(out, hidden)  # Forward pass through the vanilla RNN.
            return out, hidden  # Return the output and hidden state.
        elif this.cell_type == 'gru':
            out, hidden = this.rnn(out, hidden)  # Forward pass through the GRU.
            return out, hidden  # Return the output and hidden state.

    def init_hidden(this, batch_size):
        hidden = torch.randn((1 + int(this.bidirectional)) * this.num_layers, batch_size, this.hidden_size, device=device)  # Initialize the hidden state tensor.
        cell = torch.randn((1 + int(this.bidirectional)) * this.num_layers, batch_size, this.hidden_size, device=device)  # Initialize the cell state tensor.
        return hidden, cell


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, device, cell_type, output_vocab, embed_size, hidden_size, max_length, dropout_p=0.1, num_layers=1, bidirectional=False):
        super(DecoderRNN, self).__init__()
        # Initialize the decoder attributes
        self.hidden_size = hidden_size  # Size of the hidden state
        self.output_size = output_vocab  # Size of the output vocabulary
        self.embed_size = embed_size  # Size of the embedding
        self.dropout_p = dropout_p  # Dropout probability
        self.cell_type = cell_type  # Type of the RNN cell
        self.max_length = max_length  # Maximum length of the input sequence
        self.device = device  # Device to be used (e.g., 'cpu' or 'cuda')
        self.num_layers = num_layers  # Number of layers in the RNN
        self.embedding_decoder = nn.Embedding(self.output_size, self.embed_size)  # Embedding layer
        self.dropout = nn.Dropout(self.dropout_p)  # Dropout layer
        self.bidirectional = bidirectional  # Flag indicating bidirectional RNN

        # Initialize the RNN based on the specified cell_type
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)
        elif cell_type == 'rnn':
            self.rnn = nn.RNN(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)

        self.out = nn.Linear((1 + int(self.bidirectional)) * self.hidden_size, self.output_size)  # Linear layer
        self.out_activation = nn.LogSoftmax(dim=-1)  # Log softmax activation function

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(1)  # Add a dimension of size 1 to the input tensor
        embedded_decoder = self.embedding_decoder(input)  # Embed the input tensor
        embedded_decoder = self.dropout(embedded_decoder)  # Apply dropout to the embedded tensor

        # Pass the embedded tensor through the RNN
        if self.cell_type == 'lstm':
            output, (hidden, cell) = self.rnn(embedded_decoder, (hidden, cell))
        elif self.cell_type == 'gru':
            output, hidden = self.rnn(embedded_decoder, hidden)
        elif self.cell_type == 'rnn':
            output, hidden = self.rnn(embedded_decoder, hidden)

        output = F.relu(self.out(output))  # Apply ReLU activation to the output tensor
        output = F.log_softmax(output, dim=-1)  # Apply log softmax activation to the output tensor

        return output, hidden, cell

    def init_hidden(self, encoder_hidden, encoder_cell, encoder_bidirectional):
        hidden = encoder_hidden[-(1 + int(encoder_bidirectional)):].repeat(self.num_layers, 1, 1)  # Repeat the hidden tensor
        cell = encoder_cell[-(1 + int(encoder_bidirectional)):].repeat(self.num_layers, 1, 1)  # Repeat the cell tensor
        return hidden, cell


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(this, encoder, decoder, device):
        super().__init__()
        this.encoder = encoder
        this.decoder = decoder
        this.device = device
        this.mtl = 0  # Initialize the maximum target length
        this.sos = 0  # Initialize the start of sequence token
        
    def forward(this, source, target, teacher_forcing_ratio = 0.5):
        # Get dimensions of input
        batch_size = target.shape[0]
        target_len = target.shape[1]
        this.mtl = target_len  # Set the maximum target length
        target_vocab_size = this.decoder.output_size
        
        # Initialize tensor for storing outputs
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(this.device)

        # Initialize hidden states of the encoder
        encoder_hidden, encoder_cell = this.encoder.init_hidden(batch_size)

        # Forward pass through the encoder
        if (this.encoder.cell_type == 'lstm'):
            encoder_outputs, encoder_hidden, encoder_cell = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'rnn'):
            encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'gru'):
            encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)

        # First input to the decoder is the <sos> tokens
        input = target[:,0]
        this.sos = target[:,0]

        # Initialize hidden states of the decoder
        hidden, cell = this.decoder.init_hidden(encoder_hidden, encoder_cell, this.encoder.bidirectional)
        
        # Iterate over the target sequence
        for t in range(1, target_len):
            # Forward pass through the decoder
            output, hidden, cell = this.decoder.forward(input, hidden, cell)
            outputs[:,t] = output.squeeze(1)  # Store the decoder output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(-1)
            input = target[:,t] if teacher_force else top1.squeeze(1)

        return outputs

    def inference(this, source, target):
        # Get dimensions of input
        batch_size = source.shape[0]
        target_len = this.mtl
        target_vocab_size = this.decoder.output_size
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(this.device)

        # Initialize hidden states of the encoder
        encoder_hidden, encoder_cell = this.encoder.init_hidden(batch_size)

        # Forward pass through the encoder
        if (this.encoder.cell_type == 'lstm'):
            encoder_outputs, encoder_hidden, encoder_cell = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'rnn'):
            encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'gru'):
            encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)
           
        # First input to the decoder is the <sos> token
        input = this.sos

        # Initialize hidden states of the decoder
        hidden, cell = this.decoder.init_hidden(encoder_hidden, encoder_cell, this.encoder.bidirectional)
      
        # Iterate over the target sequence
        for t in range(1, target_len):
            # Forward pass through the decoder
            output, hidden, cell = this.decoder.forward(input, hidden, cell)
            outputs[:,t] = output.squeeze(1)  # Store the decoder output
            top1 = output.argmax(-1)
            input = top1.squeeze(1)

        # print(f"Outputs = {outputs[:,0]}")
        # print(f"Targets = {target[:,0]}")
        
        return outputs


In [ ]:
# Commented code

# Define the input and output dimensions based on the length of the dictionary
input_dim = len(dict)
output_dim = len(dict_t)

# Set the batch size for training and validation
batch_size = 32
val_batch_size = 32

# Set the embedding dimensions for the encoder and decoder
enc_embedding = 256
dec_embedding = 256

# Set the number of hidden units in the encoder and decoder
hidden = 512

# Set the number of layers in the encoder and decoder
enc_num_layers = 3
dec_num_layers = 2

# Set the dropout probabilities for the encoder and decoder
enc_dropout = 0.4
dec_dropout = 0.4

# Set the maximum length for the input sequence
max_length = mtl

# Set the cell type for the encoder and decoder
cell_type = 'gru'

# Initialize the encoder, decoder, and the overall model
enc = EncoderRNN(device, cell_type, input_dim, enc_embedding, hidden, enc_num_layers, bidirectional=True, dropout_p=enc_dropout)
dec = DecoderRNN(device, cell_type, output_dim, dec_embedding, hidden, max_length, dec_dropout, dec_num_layers, bidirectional=True)
model = Seq2Seq(enc, dec, device).to(device)

# Function to count the number of trainable parameters in the model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



# Define the optimizer and the loss criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

# Function to train the model for a given number of iterations
# Function for training the model
def trainIters(model, pairs, batch_size, n_iters, optimizer, tf, print_every=10, plot_every=10, log=True, Attention=False):
    start = time.time()  # Start time for tracking training duration
    plot_losses = []  # List to store plot losses for visualization
    train_char_accuracy = []  # List to store training character-level accuracy
    train_word_accuracy = []  # List to store training word-level accuracy
    val_losses = []  # List to store validation losses
    val_char_accuracy = []  # List to store validation character-level accuracy
    val_word_accuracy = []  # List to store validation word-level accuracy
    print_loss_total = 0  # Variable to track the cumulative print loss
    plot_loss_total = 0  # Variable to track the cumulative plot loss
    print_val_loss_total = 0  # Variable to track the cumulative validation print loss
    plot_val_loss_total = 0  # Variable to track the cumulative validation plot loss

    train_pairs = pairs[0]  # Training pairs
    val_pairs = pairs[1]  # Validation pairs
    train_accuracy = 0  # Training accuracy

    criterion = nn.NLLLoss()  # Negative Log Likelihood loss function

    count = 0  # Counter for tracking iterations
    for iter in range(1, n_iters + 1):
        for i in np.arange(start=0, stop=len(train_pairs) - batch_size, step=batch_size):
            train_accuracy = 0  # Reset training accuracy for each batch
            count += 1  # Increment the counter
            if (i + batch_size > len(train_pairs)):  # Check if the remaining examples are less than batch size
                batch_size = len(train_pairs) - i + 1  # Adjust the batch size
            input_tensor = []  # List to store input tensors for the batch
            target_tensor = []  # List to store target tensors for the batch

            for j in range(batch_size):  # Iterate over the batch examples
                input_tensor.append(train_pairs[i + j][0])  # Append input tensor to the list
                target_tensor.append(train_pairs[i + j][1])  # Append target tensor to the list

            input_tensor = torch.stack(input_tensor).squeeze(1).long().cuda()  # Convert input tensors to tensor and move to GPU
            target_tensor = torch.stack(target_tensor).squeeze(1).long().cuda()  # Convert target tensors to tensor and move to GPU

            optimizer.zero_grad()  # Zero the gradients
            if (count < 4000):  # Check if teacher forcing should be applied
                out = model(input_tensor, target_tensor, teacher_forcing_ratio=tf)  # Perform forward pass with teacher forcing
            else:
                out = model(input_tensor, target_tensor, teacher_forcing_ratio=0)  # Perform forward pass without teacher forcing

            out = torch.permute(out, [0, 2, 1])  # Permute the output tensor
            loss = criterion(out, target_tensor)  # Calculate the loss

            train_accuracy_word = word_acc(target_tensor, out) * batch_size  # Calculate word-level accuracy
            train_accuracy = train_accuracy + train_accuracy_word  # Update training accuracy
            loss.backward()  # Backpropagation
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)  # Clip gradients to prevent exploding gradients
            optimizer.step()  # Update the model parameters

            print_loss_total += loss  # Accumulate the print loss
            plot_loss_total += loss  # Accumulate the plot loss

            if count % 800 == 0:  # Perform validation every 800 iterations
                val_input_tensor = []  # List to store validation input tensors
                val_target_tensor = []  # List to store validation target tensors

                for j in range(batch_size):  # Iterate over the validation batch examples
                    val_input_tensor.append(val_pairs[j][0])  # Append validation input tensor to the list
                    val_target_tensor.append(val_pairs[j][1])  # Append validation target tensor to the list

                val_input_tensor = torch.stack(val_input_tensor).squeeze(1).long().cuda()  # Convert validation input tensors to tensor and move to GPU
                val_target_tensor = torch.stack(val_target_tensor).squeeze(1).long().cuda()  # Convert validation target tensors to tensor and move to GPU
                if (Attention == True):  # Check if attention mechanism is enabled
                    val_out, _ = model.inference(val_input_tensor, val_target_tensor)  # Perform inference with attention
                    val_out = val_out.permute(0, 2, 1)  # Permute the output tensor
                else:
                    val_out = model.inference(val_input_tensor, val_target_tensor)  # Perform inference without attention
                    val_out = val_out.permute(0, 2, 1)  # Permute the output tensor
                val_loss = criterion(val_out, val_target_tensor)  # Calculate the validation loss
                val_loss_sampled = val_loss  # Store the sampled validation loss
                wandb.log({'Val_Loss': val_loss_sampled})  # Log the validation loss

            if count % 800 == 0:  # Print and log the training loss every 800 iterations
                print_loss_avg = print_loss_total / 800  # Calculate the average print loss
                print_loss_total = 0  # Reset the print loss total
                print('%s (%d %d%%) %.7f' % (span(start, iter / n_iters), iter, iter / n_iters * 100, print_loss_avg))  # Print the progress

            if count % 800 == 0:  # Plot and log the training loss every 800 iterations
                plot_loss_avg = plot_loss_total / 800  # Calculate the average plot loss
                plot_losses.append(plot_loss_avg.detach())  # Append the plot loss to the list
                wandb.log({'Train Loss': plot_loss_avg})  # Log the training loss

                plot_loss_total = 0  # Reset the plot loss total

        train_accuracy = train_accuracy / (len(train_pairs) - batch_size)  # Calculate the average training accuracy
        train_word_accuracy.append(train_accuracy)  # Append the training accuracy to the list

    print(train_word_accuracy)  # Print the list of training word-level accuracies
    plot_losses = [losses.cpu().numpy() for losses in plot_losses]  # Convert plot losses to numpy array

    char_count = 0  # Variable to track character count
    word_count = 0  # Variable to track word count

    for i in np.arange(start=0, stop=len(val_pairs) - batch_size, step=batch_size):  # Iterate over the validation pairs
        if (i + batch_size > len(val_pairs)):  # Check if the remaining examples are less than batch size
            batch_size = len(val_pairs) - i + 1  # Adjust the batch size
        val_input_tensor = []  # List to store validation input tensors
        val_target_tensor = []  # List to store validation target tensors
        for j in range(batch_size):  # Iterate over the batch examples
            val_input_tensor.append(val_pairs[i + j][0])  # Append validation input tensor to the list
            val_target_tensor.append(val_pairs[i + j][1])  # Append validation target tensor to the list

        val_input_tensor = torch.stack(val_input_tensor).squeeze(1).long().cuda()  # Convert validation input tensors to tensor and move to GPU
        val_target_tensor = torch.stack(val_target_tensor).squeeze(1).long().cuda()  # Convert validation target tensors to tensor and move to GPU
        if (Attention == True):  # Check if attention mechanism is enabled
            val_out, _ = model.inference(val_input_tensor, val_target_tensor)  # Perform inference with attention
            val_out = val_out.permute(0, 2, 1)  # Permute the output tensor
        else:
            val_out = model.inference(val_input_tensor, val_target_tensor)  # Perform inference without attention
            val_out = val_out.permute(0, 2, 1)  # Permute the output tensor
        val_loss = criterion(val_out, val_target_tensor)  # Calculate the validation loss

        val_accuracy_word = word_acc(val_target_tensor, val_out)  # Calculate word-level accuracy
        word_count = word_count + (val_accuracy_word) * batch_size  # Update word count

    word_accuracy = word_count / (len(val_pairs) - batch_size)  # Calculate word-level accuracy

    metrics = {'Val_Accuracy': word_accuracy}  # Store validation metrics
    wandb.log(metrics)  # Log the validation metrics

    print(f"Val loss = {val_loss}")  # Print the validation loss
    print(f'Word-level-accuracy on val set = {word_accuracy}')  # Print the word-level accuracy on the validation set



In [ ]:
# Hyperparameter configuration
sweep_config = {
    'method': 'bayes',
    'name': 'Hyperparameter Tuning-Bayesian'
}

# Metric to optimize
metric = {
    'name': 'Val_Accuracy',
    'goal': 'maximize'
}

# Assign metric to sweep config
sweep_config['metric'] = metric

# Define parameter values
parameters_dict = {
    'optimiser': {
        'values': ['nadam']
    },
    'teacher_forcing_ratio': {
        'values': [0.5, 0.7]
    },
    'bidirectional': {
        'values': [True]
    },
    'enc_embedding': {
        'values': [128]
    },
    'dec_embedding': {
        'values': [128]
    },
    'epochs': {
        'values': [2]
    },
    'hidden_size': {
        'values': [64, 128, 256]
    },
    'enc_layers': {
        'values': [3]
    },
    'dec_layers': {
        'values': [3]
    },
    'dropout': {
        'values': [0.3]
    },
    'cell_type': {
        'values': ['rnn', 'gru']
    }
}

# Assign parameters to sweep config
sweep_config['parameters'] = parameters_dict

# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="CS6910 Assignment 3")


def train_sweep(config=None):
    with wandb.init(config=config) as run:
        config = wandb.config

        # Set up model parameters
        input_dim = len(dict)
        output_dim = len(dict_t)
        batch_size = 32
        val_batch_size = 32
        enc_embedding = config.enc_embedding
        dec_embedding = config.dec_embedding
        hidden = config.hidden_size
        enc_num_layers = config.enc_layers
        dec_num_layers = config.dec_layers
        enc_dropout = config.dropout
        dec_dropout = config.dropout
        max_length = mtl
        cell_type = config.cell_type

        # Initialize the encoder, decoder, and model
        enc = EncoderRNN(device, cell_type, input_dim, enc_embedding, hidden, enc_num_layers, bidirectional=config.bidirectional, dropout_p=enc_dropout)
        dec = DecoderRNN(device, cell_type, output_dim, dec_embedding, hidden, max_length, dec_dropout, dec_num_layers, bidirectional=config.bidirectional)
        model = Seq2Seq(enc, dec, device).to(device)

        # Set experiment name
        exp_name = str(config.cell_type)
        exp_name = exp_name
        exp_name = exp_name+'_optim_'+ config.optimiser
        wandb.run.name = exp_name

        # Set optimizer based on config
        if (config.optimiser == 'adam'):
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        elif (config.optimiser == 'nadam'):
            optimizer = torch.optim.NAdam(model.parameters(), lr=0.001)

        # Set loss criterion
        criterion = nn.NLLLoss()

        # Train the model
        trainIters(model, pairs, 32, config.epochs, optimizer, config.teacher_forcing_ratio)


# Run the sweep
wandb.agent(sweep_id, train_sweep, count=1)
wandb.finish()


Create sweep with ID: fxh6gson
Sweep URL: https://wandb.ai/ed22s009/CS6910%20Assignment%203/sweeps/fxh6gson


wandb: Agent Starting Run: ngfgtva3 with config:
wandb: 	bidirectional: True
wandb: 	cell_type: gru
wandb: 	dec_embedding: 128
wandb: 	dec_layers: 3
wandb: 	dropout: 0.3
wandb: 	enc_embedding: 128
wandb: 	enc_layers: 3
wandb: 	epochs: 2
wandb: 	hidden_size: 128
wandb: 	optimiser: nadam
wandb: 	teacher_forcing_ratio: 0.5
wandb: Currently logged in as: ed22s009. Use `wandb login --relogin` to force relogin


[0.0, 0.0]
Val loss = 1.1140252351760864
Word-level-accuracy on val set = 0.0


Val_Accuracy,▁
Val_Accuracy,0.0


In [ ]:
# Define a function to append strings to a CSV file
def append_strings_to_csv(file_path, string1, string2, string3):
    with open(file_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([string1, string2, string3])

# Calculate the input and output dimensions based on a dictionary
input_dim = len(dict)
output_dim = len(dict_t)

# Set batch sizes, embedding dimensions, hidden sizes, and other parameters
batch_size = 32
val_batch_size = 32
enc_embedding = 128
dec_embedding = 128
hidden = 512
epochs = 8
enc_num_layers = 3
dec_num_layers = 3
enc_dropout = 0.3
dec_dropout = 0.3
max_length = mtl
cell_type = 'lstm'
teacher_forcing_ratio = 0

# Create encoder, decoder, and Seq2Seq model objects
enc = EncoderRNN(device, cell_type, input_dim, enc_embedding, hidden, enc_num_layers, bidirectional=True, dropout_p=enc_dropout)
dec = DecoderRNN(device, cell_type, output_dim, dec_embedding, hidden, max_length, dec_dropout, dec_num_layers, bidirectional=True)
model = Seq2Seq(enc, dec, device).to(device)

# Set optimizer and learning rate
optimizer = torch.optim.NAdam(model.parameters(), lr=0.001)

# Initialize Weights and Biases (wandb)
wandb.init()

# Train the model using the defined training function
trainIters(model, pairs, 32, epochs, optimizer, teacher_forcing_ratio)

# Select test pairs for evaluation
test_pairs = pairs[2]
test_word_count = 0

# Set the initial count and name for the output file
count = 1
name = 'AttentionPredictions' + str(count)

# Define the file path for saving the CSV file
file_path = '/content/drive/MyDrive/aksharantar_sampled/mar' + name + '.csv'

# Iterate over the test pairs in batches
for i in np.arange(start=0, stop=len(test_pairs) - batch_size, step=batch_size):
    if (i + batch_size > len(test_pairs)):
        batch_size = len(test_pairs) - i + 1

    # Initialize empty lists for storing input and target tensors
    test_input_tensor = []
    test_target_tensor = []

    # Populate the input and target tensors for the current batch
    for j in range(batch_size):
        test_input_tensor.append(test_pairs[i + j][0])
        test_target_tensor.append(test_pairs[i + j][1])

    # Convert the input and target tensors to the appropriate format and move them to the GPU
    test_input_tensor = torch.stack(test_input_tensor).squeeze(1).long().cuda()
    test_target_tensor = torch.stack(test_target_tensor).squeeze(1).long().cuda()

    # Perform inference using the model on the test input tensor
    test_out = model.inference(test_input_tensor, test_target_tensor)

    # Permute the dimensions of the output tensor
    test_out = test_out.permute(0, 2, 1)

    # Calculate the test loss
    test_loss = criterion(test_out, test_target_tensor)

    # Iterate over the batch and append the input, target, and output strings to the CSV file
    for j in range(batch_size):
        input_str = key_presenting(dict, test_input_tensor[j])
        output_str = key_presenting(dict_t, torch.argmax(test_out[j], dim=0))
        target_str = key_presenting(dict_t, test_target_tensor[j])
        append_strings_to_csv(file_path, input_str, target_str, output_str)

    # Calculate the word accuracy for the test batch
    test_accuracy_word = word_acc(test_target_tensor, test_out)
    test_word_count = test_word_count + (test_accuracy_word) * batch_size

# Calculate the overall word accuracy for the test set
test_word_accuracy = test_word_count / (len(test_pairs) - batch_size)

# Print the accuracy on the test set
print(f"Accuracy on Test Set is {test_word_accuracy}")


1m 27s (- 0m 12s) (7 87%) 1.0391052
[0.0, 0.0, 0.0, 0.0, 0.0002461235540241201, 0.0, 0.0014767413241447206, 0.0014767413241447206]
Val loss = 0.7258210778236389
Word-level-accuracy on val set = 0.04331774550824514
Accuracy on Test Set is 0.1553039625892198


In [ ]:
class AttentionDecoderRNN(nn.Module):
    def __init__(this, device, cell_type, output_vocab, embed_size, hidden_size, max_length, dropout_p=0.1, num_layers = 1, bidirectional = False):
        super(AttentionDecoderRNN, this).__init__()
        this.hidden_size = hidden_size  # Initialize hidden size
        this.output_size = output_vocab  # Initialize output vocabulary size
        this.embed_size = embed_size  # Initialize embedding size
        this.dropout_p = dropout_p  # Initialize dropout probability
        this.cell_type = cell_type  # Initialize cell type
        this.max_length = max_length  # Initialize maximum length
        this.device = device  # Initialize device
        this.num_layers = num_layers  # Initialize number of layers
        this.embedding_decoder = nn.Embedding(this.output_size, this.embed_size)  # Initialize decoder embedding
        this.dropout = nn.Dropout(this.dropout_p)  # Initialize dropout layer
        this.bidirectional = bidirectional  # Initialize bidirectional flag

        # Initialize the appropriate RNN cell based on the given cell type
        if (cell_type == 'lstm'):
          this.rnn = nn.LSTM( this.embed_size + hidden_size*(1+int(this.bidirectional)), hidden_size, num_layers = this.num_layers, batch_first=True, bidirectional=this.bidirectional, dropout = this.dropout_p)
        elif (cell_type == 'gru'):
          this.rnn = nn.GRU(this.embed_size+ hidden_size*(1+int(this.bidirectional)), hidden_size, num_layers = this.num_layers, batch_first=True, bidirectional=this.bidirectional, dropout = this.dropout_p)
        elif (cell_type == 'rnn'):
          this.rnn = nn.RNN(this.embed_size+ hidden_size*(1+int(this.bidirectional)), hidden_size, num_layers = this.num_layers, batch_first=True, bidirectional=this.bidirectional, dropout = this.dropout_p)

        # Initialize attention layers
        this.energy = nn.Linear(hidden_size*(2+int(this.bidirectional)),hidden_size)
        this.value = nn.Linear(hidden_size,1, bias = False)
        this.softmax = nn.Softmax(dim = 0)
        this.relu = nn.ReLU()
        this.tanh = nn.Tanh()

        # Initialize output layers
        this.out = nn.Linear((1+int(this.bidirectional))*this.hidden_size, this.output_size)
        this.out_activation = nn.LogSoftmax(dim=-1)

        # Initialize hidden reshape layer
        this.hidden_reshape_linear = nn.Linear(hidden_size*2,hidden_size)

    def forward(this, input, encoder_states, hidden, cell):
        input = input.unsqueeze(1)  # Add a dimension to the input tensor
        embedded_decoder = this.embedding_decoder(input)  # Embed the input tensor
        embedded_decoder = this.dropout(embedded_decoder)  # Apply dropout to the embedded tensor

        encoder_states = encoder_states.permute(1,0,2)  # Permute the encoder states tensor
        sequence_length = encoder_states.shape[0]  # Get the sequence length
        if this.bidirectional == True:
          hidden_1 = this.relu(this.hidden_reshape_linear(hidden[0:2].permute(1,0,2).reshape(hidden.shape[1],-1))).unsqueeze(0)  # Reshape and apply a linear layer to the hidden tensor
        else:
          hidden_1 = hidden[0]  # Use the hidden tensor as is

        hidden_reshaped = hidden_1.repeat(sequence_length,1,1)  # Repeat the hidden tensor based on the sequence length

        energy = this.value(this.tanh(this.energy(torch.cat((hidden_reshaped,encoder_states),dim=2))))  # Calculate the attention energy
        attention = this.softmax(energy)  # Apply softmax to the energy tensor
        attention = attention.permute(1,2,0)  # Permute the attention tensor
        encoder_states = encoder_states.permute(1,0,2)  # Permute the encoder states tensor
        context_vector = torch.bmm(attention, encoder_states)  # Calculate the context vector using attention and encoder states

        rnn_input = torch.cat((context_vector,embedded_decoder), dim = 2)  # Concatenate the context vector and embedded decoder tensor

        if (this.cell_type == 'lstm'):
          decoder_output, (hidden, cell) = this.rnn(rnn_input, (hidden, cell))  # Apply the RNN to the input tensor
        elif (this.cell_type == 'gru'):
          decoder_output, hidden = this.rnn(rnn_input, hidden)  # Apply the RNN to the input tensor
        elif (this.cell_type == 'rnn'):
          decoder_output,hidden = this.rnn(rnn_input, hidden)  # Apply the RNN to the input tensor

        output = F.relu(this.out(decoder_output))  # Apply ReLU activation to the output tensor
        output = F.log_softmax(output,dim=-1)  # Apply log softmax activation to the output tensor

        return output, hidden, cell, attention  # Return the output, hidden, cell, and attention tensors

    def init_hidden(this, encoder_hidden, encoder_cell, encoder_bidirectional):
        hidden = encoder_hidden[-(1+int(encoder_bidirectional)): ].repeat(this.num_layers,1,1)  # Repeat the encoder hidden tensor based on the number of layers
        cell = encoder_cell[-(1+int(encoder_bidirectional)): ].repeat(this.num_layers,1,1)  # Repeat the encoder cell tensor based on the number of layers
        return hidden, cell  # Return the hidden and cell tensors


In [ ]:
class AttentionSeq2Seq(nn.Module):
    def __init__(this, encoder, decoder, device):
        super().__init__()
        this.encoder = encoder  # Initialize the encoder
        this.decoder = decoder  # Initialize the decoder
        this.device = device    # Set the device (CPU or GPU)
        this.mtl = 0            # Initialize the maximum target length
        this.sos = 0            # Initialize the start of sequence token

    def forward(this, source, target, teacher_forcing_ratio = 0.5):
        # Initialize variables
        batch_size = target.shape[0]      # Get the batch size
        target_len = target.shape[1]      # Get the target sequence length
        this.mtl = target_len             # Update the maximum target length
        target_vocab_size = this.decoder.output_size  # Get the size of the target vocabulary
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(this.device)  # Initialize the outputs tensor
        encoder_hidden, encoder_cell = this.encoder.init_hidden(batch_size)  # Initialize the encoder hidden state and cell state

        # Forward pass through the encoder
        if (this.encoder.cell_type == 'lstm'):
          encoder_outputs, encoder_hidden, encoder_cell = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'rnn'):
          encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'gru'):
          encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)

        # Initialize decoder input
        input = target[:, 0]    # Get the first target token
        this.sos = target[:, 0]  # Update the start of sequence token
        hidden, cell = this.decoder.init_hidden(encoder_hidden, encoder_cell, this.encoder.bidirectional)  # Initialize the decoder hidden state and cell state
        
        # Decode sequence
        for t in range(1, target_len):
            output, hidden, cell, _ = this.decoder.forward(input, encoder_outputs, hidden, cell)  # Perform a forward pass through the decoder
            outputs[:, t] = output.squeeze(1)  # Store the output in the outputs tensor
            teacher_force = random.random() < teacher_forcing_ratio  # Determine if teacher forcing will be used
            top1 = output.argmax(-1)  # Get the index of the highest probability token
            input = target[:, t] if teacher_force else top1.squeeze(1)  # Update the decoder input based on teacher forcing

        return outputs

    def inference(this, source, target):
        # Initialize variables
        batch_size = source.shape[0]  # Get the batch size
        target_len = this.mtl         # Get the maximum target length
        target_vocab_size = this.decoder.output_size  # Get the size of the target vocabulary
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(this.device)  # Initialize the outputs tensor
        encoder_hidden, encoder_cell = this.encoder.init_hidden(batch_size)  # Initialize the encoder hidden state and cell state

        # Forward pass through the encoder
        if (this.encoder.cell_type == 'lstm'):
          encoder_outputs, encoder_hidden, encoder_cell = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'rnn'):
          encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)
        if (this.encoder.cell_type == 'gru'):
          encoder_outputs, encoder_hidden = this.encoder.forward(source, encoder_hidden, encoder_cell)

        # Initialize decoder input
        input = this.sos                    # Get the start of sequence token
        input_len = encoder_outputs.shape[1]  # Get the input sequence length
        hidden, cell = this.decoder.init_hidden(encoder_hidden, encoder_cell, this.encoder.bidirectional)  # Initialize the decoder hidden state and cell state
        attention_map = torch.zeros(batch_size, target_len, input_len)  # Initialize the attention map tensor

        for t in range(1, target_len):
            output, hidden, cell, attention = this.decoder.forward(input, encoder_outputs, hidden, cell)  # Perform a forward pass through the decoder
            attention_map[:, t-1, :] = attention.squeeze(1)  # Store the attention weights in the attention map tensor
            outputs[:, t] = output.squeeze(1)  # Store the output in the outputs tensor
            top1 = output.argmax(-1)  # Get the index of the highest probability token
            input = top1.squeeze(1)  # Update the decoder input with the predicted token

        return outputs, attention_map


In [ ]:
# Define the sweep configuration for Bayesian optimization
sweep_config = {
    'method': 'bayes',
    'name': 'Hyperparameter Tuning-Bayesian'
}

# Define the metric to be maximized during tuning
metric = {
    'name': 'Val_Accuracy',
    'goal': 'maximize'   
}

# Add the metric to the sweep configuration
sweep_config['metric'] = metric

# Define the dictionary of hyperparameters and their values to be tuned
parameters_dict = {
    'optimiser': {
        'values': ['nadam']
    },
    'teacher_forcing_ratio': {
        'values': [0.5, 0.7]
    },
    'bidirectional': {
        'values': [True, False]
    },
    'enc_embedding': {
        'values': [128, 256]
    },
    'dec_embedding': {
        'values': [128, 256]
    },
    'epochs': {
        'values': [3, 5]
    },
    'hidden_size': {
        'values': [64, 128, 256]
    },
    'enc_layers': {
        'values': [2, 3]
    },
    'dec_layers': {
        'values': [2, 3]
    },
    'dropout': {
        'values': [0.25, 0.4]
    },
    'cell_type': {
        'values': ['gru', 'rnn']
    }
}

# Add the hyperparameter dictionary to the sweep configuration
sweep_config['parameters'] = parameters_dict

# Initialize the sweep with the given configuration and project name
sweep_id = wandb.sweep(sweep_config, project="Attention Assignment 3")

# Function to train the model with different configurations
def train_sweep(config=None):
    with wandb.init(config=config) as run:
        config = wandb.config

        # Set up the model and hyperparameters
        input_dim = len(dict)
        output_dim = len(dict_t)
        batch_size = 32
        val_batch_size = 32
        enc_embedding = config.enc_embedding
        dec_embedding = config.dec_embedding
        hidden = config.hidden_size
        enc_num_layers = config.enc_layers
        dec_num_layers = config.dec_layers
        enc_dropout = config.dropout
        dec_dropout = config.dropout
        max_length = mtl
        cell_type = config.cell_type

        # Create encoder and decoder instances
        enc = EncoderRNN(device, cell_type, input_dim, enc_embedding, hidden, enc_num_layers, bidirectional=config.bidirectional, dropout_p=enc_dropout)
        dec = AttentionDecoderRNN(device, cell_type, output_dim, dec_embedding, hidden, max_length, dec_dropout, dec_num_layers, bidirectional=config.bidirectional)
        model = AttentionSeq2Seq(enc, dec, device).to(device)

        # Determine the model name based on bidirectional and cell type
        if config.bidirectional:
            m = '_bi_'
        else:
            m = '_uni_'

        exp_name = str(config.cell_type)
        exp_name = exp_name
        exp_name = exp_name + '_optim_' + config.optimiser

        wandb.run.name = exp_name

        # Choose the optimizer based on the configuration
        if config.optimiser == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        elif config.optimiser == 'nadam':
            optimizer = torch.optim.NAdam(model.parameters(), lr=0.001)

        criterion = nn.NLLLoss()

        # Train the model with the given hyperparameters
        trainIters(model, pairs, 32, config.epochs, optimizer, config.teacher_forcing_ratio, Attention=True)

# Run the sweep agent to train the model with different configurations
wandb.agent(sweep_id, train_sweep, count=50)

# Finish the run
wandb.finish()


Create sweep with ID: 3d3bggif
Sweep URL: https://wandb.ai/ed22s009/Attention%20Assignment%203/sweeps/3d3bggif


wandb: Agent Starting Run: p5y9khrm with config:
wandb: 	bidirectional: False
wandb: 	cell_type: rnn
wandb: 	dec_embedding: 256
wandb: 	dec_layers: 2
wandb: 	dropout: 0.4
wandb: 	enc_embedding: 128
wandb: 	enc_layers: 3
wandb: 	epochs: 3
wandb: 	hidden_size: 128
wandb: 	optimiser: nadam
wandb: 	teacher_forcing_ratio: 0.5
wandb: Currently logged in as: ed22s009. Use `wandb login --relogin` to force relogin


[0.0, 0.0, 0.0]
Val loss = 1.3782434463500977
Word-level-accuracy on val set = 0.0


Val_Accuracy,▁
Val_Accuracy,0.0


wandb: Agent Starting Run: v6tf2y2p with config:
wandb: 	bidirectional: False
wandb: 	cell_type: rnn
wandb: 	dec_embedding: 128
wandb: 	dec_layers: 2
wandb: 	dropout: 0.25
wandb: 	enc_embedding: 256
wandb: 	enc_layers: 2
wandb: 	epochs: 5
wandb: 	hidden_size: 64
wandb: 	optimiser: nadam
wandb: 	teacher_forcing_ratio: 0.7
